# Travailler avec les paires clefs-valeurs


## Chargement des données

In [222]:
num1 = sc.parallelize([1,1,1,2,3,4,5,5,6,7,8,8,8])
num2 = sc.parallelize(xrange(4,11))
txt1 = sc.parallelize(['allo les amis', 'ne riez pas', 'de mon apprentissage'])
txt2 = sc.textFile("/etc/passwd")
txt3 = sc.parallelize(["BRLAV35;Lavoie;Bruno","FRPOL9;Poliquin;Frederic","DOASS4;Asselin;Dominique","ALWHI3;Whittom;Allen"])

##  Créer des pairs-RDDs

In [223]:
# Créer une paire numéro --> valeur au carré
pair1 = num1.map(lambda x: (x, x*x))
pair1.collect()

[(1, 1),
 (1, 1),
 (1, 1),
 (2, 4),
 (3, 9),
 (4, 16),
 (5, 25),
 (5, 25),
 (6, 36),
 (7, 49),
 (8, 64),
 (8, 64),
 (8, 64)]

In [224]:
# Créer une paire numéro --> valeur au cube
pair2 = num2.map(lambda x: (x, x^3))
pair2.collect()

[(4, 7), (5, 6), (6, 5), (7, 4), (8, 11), (9, 10), (10, 9)]

In [225]:
# Créer une paire du type code_utilisateur -> informations
pair3 = txt3.map(lambda x: (x.split(";", 1)[0], x.split(";", 1)[1]))
pair3.collect()

[('BRLAV35', 'Lavoie;Bruno'),
 ('FRPOL9', 'Poliquin;Frederic'),
 ('DOASS4', 'Asselin;Dominique'),
 ('ALWHI3', 'Whittom;Allen')]

## Transformations sur les pairs-RDDs

### filter(func)

In [226]:
# just for fun
pair1.filter(lambda x: x[0] > 5).collect()

[(6, 36), (7, 49), (8, 64), (8, 64), (8, 64)]

### reduceByKey(func)
__Utilité:__ Fusionne ensemble les valeurs de chaque clef en utilisant une fonction associative.

In [227]:
pair1.reduceByKey(lambda x,y: x + y).collect()

[(8, 192), (4, 16), (1, 3), (5, 50), (2, 4), (6, 36), (3, 9), (7, 49)]

In [228]:
# On peut voir que la fonction ne peut pas être appelée s'il y a seulement qu'un élément pour une clef.
pair1.reduceByKey(lambda x,y: x>0).collect()

[(8, True), (4, 16), (1, True), (5, True), (2, 4), (6, 36), (3, 9), (7, 49)]

### groupByKey()
__Utilité:__ Groupe les valeurs qui ont la même clef. Ressemble drôlement un à un GROUP BY + COLLECT en SQL.

In [229]:
# retourne un itérable...
pair1.groupByKey().collect()

[(8, <pyspark.resultiterable.ResultIterable at 0x7f1b94474e90>),
 (4, <pyspark.resultiterable.ResultIterable at 0x7f1b94474c90>),
 (1, <pyspark.resultiterable.ResultIterable at 0x7f1b94474190>),
 (5, <pyspark.resultiterable.ResultIterable at 0x7f1b94474310>),
 (2, <pyspark.resultiterable.ResultIterable at 0x7f1b94474f50>),
 (6, <pyspark.resultiterable.ResultIterable at 0x7f1b94474a90>),
 (3, <pyspark.resultiterable.ResultIterable at 0x7f1b94474cd0>),
 (7, <pyspark.resultiterable.ResultIterable at 0x7f1b94474fd0>)]

In [230]:
# petit truc pour visualiser, on fait un map sur les valeurs (mapValues)...
pair1.groupByKey().mapValues(list).collect()

[(8, [64, 64, 64]),
 (4, [16]),
 (1, [1, 1, 1]),
 (5, [25, 25]),
 (2, [4]),
 (6, [36]),
 (3, [9]),
 (7, [49])]

### combineByKey(createCombiner, mergeValue, mergeCombiners)
__Utilité:__ Combine les valeurs ayant la même clef, en retournant un type différent.

Generic function to combine the elements for each key using a custom set of aggregation functions.

Turns an RDD[(K, V)] into a result of type RDD[(K, C)], for a “combined type” C. Note that V and C can be different – for example, one might group an RDD of type (Int, Int) into an RDD of type (Int, List[Int]).

Users provide three functions:

- __createCombiner__, which turns a V into a C (e.g., creates a one-element list)
- __mergeValue__, to merge a V into a C (e.g., adds it to the end of a list)
- __mergeCombiners__, to combine two C’s into a single one.

Lien: http://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=sample#pyspark.RDD.combineByKey

In [231]:
pair1.collect()

[(1, 1),
 (1, 1),
 (1, 1),
 (2, 4),
 (3, 9),
 (4, 16),
 (5, 25),
 (5, 25),
 (6, 36),
 (7, 49),
 (8, 64),
 (8, 64),
 (8, 64)]

In [232]:
# Convertir le type comme c'est réellement l'intention de la fonction
# Aucun + ne devrait apparaitre en local si nous n'avons pas plus d'une partitions.

pair1.combineByKey(lambda x: str(x), 
                   lambda x,y: x + '*' + str(y), 
                   lambda x,y: x + '+' + y).collect()


[(8, '64*64*64'),
 (4, '16'),
 (1, '1*1*1'),
 (5, '25*25'),
 (2, '4'),
 (6, '36'),
 (3, '9'),
 (7, '49')]

In [233]:
# On peut émuler un groupByKey

pair1.combineByKey(lambda v:     [v], 
                   lambda c,v:   c + [v], 
                   lambda c1,c2: c1 + c2).collect()


[(8, [64, 64, 64]),
 (4, [16]),
 (1, [1, 1, 1]),
 (5, [25, 25]),
 (2, [4]),
 (6, [36]),
 (3, [9]),
 (7, [49])]

### mapValues(func)
__Utilité:__ appliquer une fonction à chaque valeur sans changer la clef.

In [234]:
pair1.mapValues(lambda x: 'x' + str(x*2)).collect()

[(1, 'x2'),
 (1, 'x2'),
 (1, 'x2'),
 (2, 'x8'),
 (3, 'x18'),
 (4, 'x32'),
 (5, 'x50'),
 (5, 'x50'),
 (6, 'x72'),
 (7, 'x98'),
 (8, 'x128'),
 (8, 'x128'),
 (8, 'x128')]

### flatMapValues(func)
__Utilité:__ appliquer une fonction qui retourne un itérateur pour chaque valeur d'un pair-RDD et chaque élément retourné produit une entrée clef-valeur avec la vieille clef. Souvent utilisé pour tokenization.

In [235]:
pair3.collect()

[('BRLAV35', 'Lavoie;Bruno'),
 ('FRPOL9', 'Poliquin;Frederic'),
 ('DOASS4', 'Asselin;Dominique'),
 ('ALWHI3', 'Whittom;Allen')]

In [236]:
pair3.flatMapValues(lambda x: x.split(';')).collect()

[('BRLAV35', 'Lavoie'),
 ('BRLAV35', 'Bruno'),
 ('FRPOL9', 'Poliquin'),
 ('FRPOL9', 'Frederic'),
 ('DOASS4', 'Asselin'),
 ('DOASS4', 'Dominique'),
 ('ALWHI3', 'Whittom'),
 ('ALWHI3', 'Allen')]

In [237]:
# Peut aussi être utile pour pivoter les valeurs qui sont des listes
# Créer les listes en valeurs...
temp = pair3.mapValues(lambda x: x.split(';'))
temp.collect()

[('BRLAV35', ['Lavoie', 'Bruno']),
 ('FRPOL9', ['Poliquin', 'Frederic']),
 ('DOASS4', ['Asselin', 'Dominique']),
 ('ALWHI3', ['Whittom', 'Allen'])]

In [238]:
# Pivoter...
temp.flatMapValues(list).collect()

[('BRLAV35', 'Lavoie'),
 ('BRLAV35', 'Bruno'),
 ('FRPOL9', 'Poliquin'),
 ('FRPOL9', 'Frederic'),
 ('DOASS4', 'Asselin'),
 ('DOASS4', 'Dominique'),
 ('ALWHI3', 'Whittom'),
 ('ALWHI3', 'Allen')]

###  keys()

In [239]:
pair3.keys().collect()

['BRLAV35', 'FRPOL9', 'DOASS4', 'ALWHI3']

### values()

In [240]:
pair3.values().collect()

['Lavoie;Bruno', 'Poliquin;Frederic', 'Asselin;Dominique', 'Whittom;Allen']

### sortByKeys()

In [241]:
pair3.sortByKey().collect()

[('ALWHI3', 'Whittom;Allen'),
 ('BRLAV35', 'Lavoie;Bruno'),
 ('DOASS4', 'Asselin;Dominique'),
 ('FRPOL9', 'Poliquin;Frederic')]

### Transformations sur 2 pair-RDDs

In [242]:
pair1.collect()

[(1, 1),
 (1, 1),
 (1, 1),
 (2, 4),
 (3, 9),
 (4, 16),
 (5, 25),
 (5, 25),
 (6, 36),
 (7, 49),
 (8, 64),
 (8, 64),
 (8, 64)]

In [243]:
pair2.collect()

[(4, 7), (5, 6), (6, 5), (7, 4), (8, 11), (9, 10), (10, 9)]

### subtractByKey()

In [244]:
pair1.subtractByKey(pair2).collect()

[(1, 1), (1, 1), (1, 1), (2, 4), (3, 9)]

### join()

In [245]:
pair1.join(pair2).collect()

[(8, (64, 11)),
 (8, (64, 11)),
 (8, (64, 11)),
 (4, (16, 7)),
 (5, (25, 6)),
 (5, (25, 6)),
 (6, (36, 5)),
 (7, (49, 4))]

### rightOuterJoin() & leftOuterJoin()

In [246]:
pair1.rightOuterJoin(pair2).collect()

[(8, (64, 11)),
 (8, (64, 11)),
 (8, (64, 11)),
 (9, (None, 10)),
 (10, (None, 9)),
 (4, (16, 7)),
 (5, (25, 6)),
 (5, (25, 6)),
 (6, (36, 5)),
 (7, (49, 4))]

In [247]:
pair1.leftOuterJoin(pair2).collect()

[(8, (64, 11)),
 (8, (64, 11)),
 (8, (64, 11)),
 (1, (1, None)),
 (1, (1, None)),
 (1, (1, None)),
 (2, (4, None)),
 (3, (9, None)),
 (4, (16, 7)),
 (5, (25, 6)),
 (5, (25, 6)),
 (6, (36, 5)),
 (7, (49, 4))]

### cogroup()
__Utilité:__ groupe les données des deux RDD partageant les mêmes clefs.

In [248]:
pair1.cogroup(pair2).collect()

[(8,
  (<pyspark.resultiterable.ResultIterable at 0x7f1b94409e50>,
   <pyspark.resultiterable.ResultIterable at 0x7f1b9444e7d0>)),
 (1,
  (<pyspark.resultiterable.ResultIterable at 0x7f1b9444e510>,
   <pyspark.resultiterable.ResultIterable at 0x7f1b9444e610>)),
 (9,
  (<pyspark.resultiterable.ResultIterable at 0x7f1b9444ef90>,
   <pyspark.resultiterable.ResultIterable at 0x7f1b9444e650>)),
 (2,
  (<pyspark.resultiterable.ResultIterable at 0x7f1b9444ed90>,
   <pyspark.resultiterable.ResultIterable at 0x7f1b9444e710>)),
 (10,
  (<pyspark.resultiterable.ResultIterable at 0x7f1b943821d0>,
   <pyspark.resultiterable.ResultIterable at 0x7f1b94382950>)),
 (3,
  (<pyspark.resultiterable.ResultIterable at 0x7f1b94382590>,
   <pyspark.resultiterable.ResultIterable at 0x7f1b943825d0>)),
 (4,
  (<pyspark.resultiterable.ResultIterable at 0x7f1b943828d0>,
   <pyspark.resultiterable.ResultIterable at 0x7f1b94382390>)),
 (5,
  (<pyspark.resultiterable.ResultIterable at 0x7f1b94382090>,
   <pyspark.res

### countByKey() & countByValue()


In [249]:
pair1.countByKey()

defaultdict(<type 'int'>, {1: 3, 2: 1, 3: 1, 4: 1, 5: 2, 6: 1, 7: 1, 8: 3})

In [250]:
pair1.countByValue()

defaultdict(<type 'int'>, {(6, 36): 1, (7, 49): 1, (8, 64): 3, (4, 16): 1, (5, 25): 2, (3, 9): 1, (2, 4): 1, (1, 1): 3})

In [251]:
pair1.lookup(4)

[16]